<a href="https://colab.research.google.com/github/MarioPrado1148/DSWP_Aluno_Mario/blob/MarioPrado1148-Notebooks/Kaggle_catboost_20201109_passo_a_passo_Zenilson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Instalação do CATBOOST


In [ ]:
!pip install catboost

     |████████████████████████████████| 66.2MB 57kB/s 


##2. Importação das bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#from sklearn.metrics import confusion_matrix # para plotar a confusion matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier



## 3. Carregando os dataframes iniciais com os dados de treino e teste

In [ ]:
df_treino = pd.read_csv("train.csv")
df_teste = pd.read_csv("test.csv")


## 4. Classe para tratamento dos dados do DataFrame.   A intenção ao utilizar uma classe pra isso foi a possibilidade de ir adicionando métodos que retornem diferentes "versões" do dataframe, com diferentes tratamentos de features...e ir testando/mantendo aquelas que tenham mais sucesso.
### Observações:
1. O dataframe "original" é passado como parâmetro na criação do objeto.   Ao instanciar o objeto alguns tratamentos iniciais, sugeridos pelo Mario, são aplicados (ex: renomear colunas, missing values, conversão de tipo, etc.) 
2. Após a criação, posso utilizar algo como objeto.getDataFramexxxxxx() para obter o dataframe com o tratamento específico definido no respectivo método.

In [ ]:
class Trata_DataFrame():
    def __init__(self,df):
      self.df = df
      self.renomeia_colunas()
      self.trata_CobrancaTotal()
      self.trata_MesesNaCompanhia()
      self.trata_MetodoPagamento()
      self.trata_TemDependentes()

    def renomeia_colunas(self):
       self.df.rename(columns = {'id':'id',
                                'gender':'genero',
                                'SeniorCitizen':'Idoso',
                                'Partner':'TemParceiro',
                                'Dependents':'TemDependentes',
                                'tenure':'MesesNaCompanhia',
                                'PhoneService':'TemTelefone',
                                'MultipleLines':'MaisLinhas',
                                'InternetService':'TemInternet',
                                'OnlineSecurity':'ServicoSegurancaOnline',
                                'OnlineBackup':'ServicoBackupOnline',
                                'DeviceProtection':'ServicoProtecaoOnline',
                                'TechSupport':'ServicoSuporteOnline',
                                'StreamingTV':'ServicoTV',
                                'StreamingMovies':'ServicoPctFilmes',
                                'Contract':'Contrato',
                                'PaperlessBilling':'ContaOnline',
                                'PaymentMethod':'MetodoPagamento',
                                'MonthlyCharges':'CobrancaMensal',
                                'TotalCharges':'CobrancaTotal',
                                'Churn':'Churn'}, inplace=True)   

    def trata_MetodoPagamento(self):
        ''' exclui as linhas que estão com valores nulos, por serem irrelevantes...'''
        #self.df = self.df.dropna(axis = 0, subset = ['MetodoPagamento'])
        self.df.loc[self.df["MetodoPagamento"].isna(),"MetodoPagamento"] = "MISS"
    def trata_TemDependentes(self):
        ''' exclui as linhas que estão com valores nulos, por serem irrelevantes...'''
        self.df.loc[self.df["TemDependentes"].isna(),"TemDependentes"] = "MISS"
        self.df.loc[(self.df["MaisLinhas"] == 'No') & (self.df["TemDependentes"] == 'MISS'), "TemDependentes"] = "No"
        self.df.loc[(self.df["MaisLinhas"] == 'Yes') & (self.df["TemDependentes"] == 'MISS'), "TemDependentes"] = "Yes"
        #self.df.loc[np.isnan(self.df["TemDependentes"]),"TemDependentes"] = "MISS" #poderia atribuir yes para quem tiver mais de uma linha...

    def trata_CobrancaTotal(self):
        '''  converte os valores para número...Em seguida substitui Nan por MesesNaCompanhia * CobrancaMensal '''
        self.df['CobrancaTotal'] = pd.to_numeric(self.df['CobrancaTotal'], errors='coerce')
        self.df.loc[np.isnan(self.df["CobrancaTotal"]),"CobrancaTotal"] = self.df["MesesNaCompanhia"] * self.df["CobrancaMensal"]

    def trata_MesesNaCompanhia(self):
        ''' recalcula a qtde de meses dos missing values dividindo o valor total pelo valor mensal '''   
        self.df.loc[np.isnan(self.df["MesesNaCompanhia"]),"MesesNaCompanhia"] = self.df["CobrancaTotal"] / self.df["CobrancaMensal"]

    def getDataFrameDefault(self):
        ''' retorna um dataframe com as transformações "padrão"'''
        return self.df  

    def getDataFrameMelhoresFeatures(self):
        ''' retorna o dataframe default, adicionando duas novas colunas (qtdeServicos e vlrMedioPorServico), e igualando "No" e "No internet service" das colunas relativas a serviços online '''
        df2 = self.df.copy()
        df2["QtdeServicosAdicionaisInternet"] = 0
        df2.loc[df2["ServicoSegurancaOnline"] == "Yes","QtdeServicosAdicionaisInternet"] = df2["QtdeServicosAdicionaisInternet"] + 1
        df2.loc[df2["ServicoBackupOnline"] == "Yes","QtdeServicosAdicionaisInternet"] =  df2["QtdeServicosAdicionaisInternet"] + 1
        df2.loc[df2["ServicoProtecaoOnline"] == "Yes","QtdeServicosAdicionaisInternet"] =  df2["QtdeServicosAdicionaisInternet"] + 1
        df2.loc[df2["ServicoSuporteOnline"] == "Yes","QtdeServicosAdicionaisInternet"] = df2["QtdeServicosAdicionaisInternet"] + 1

        df2.loc[df2["ServicoTV"] != "Yes","ServicoTV"] =  "No"
        df2.loc[df2["ServicoPctFilmes"] != "Yes","ServicoPctFilmes"] = "No"
        df2["QtdeServicosAdicionaisInternet"] = df2["QtdeServicosAdicionaisInternet"].astype('float')
        df2.drop(columns=["genero","TemParceiro","TemTelefone","ServicoSegurancaOnline","ServicoBackupOnline","ServicoProtecaoOnline","ServicoSuporteOnline" ],inplace=True)
        return df2         

    def getDataFramePrecoMedio(self):
        ''' retorna o dataframe default, adicionando duas novas colunas (qtdeServicos e vlrMedioPorServico), e igualando "No" e "No internet service" das colunas relativas a serviços online '''
        df2 = self.df.copy()
        df2["QtdeServicos"] = 0
        df2.loc[df2["TemTelefone"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["TemInternet"] != "No","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoTV"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoSegurancaOnline"] == "Yes","QtdeServicos"] = df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoBackupOnline"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoProtecaoOnline"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoPctFilmes"] == "Yes","QtdeServicos"] = df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoSuporteOnline"] == "Yes","QtdeServicos"] = df2["QtdeServicos"] + 1

        df2.loc[df2["ServicoTV"] != "Yes","ServicoTV"] =  "No"
        df2.loc[df2["ServicoSegurancaOnline"] != "Yes","ServicoSegurancaOnline"] = "No"
        df2.loc[df2["ServicoBackupOnline"] != "Yes","ServicoBackupOnline"] =  "No"
        df2.loc[df2["ServicoProtecaoOnline"] != "Yes","ServicoProtecaoOnline"] = "No"
        df2.loc[df2["ServicoPctFilmes"] != "Yes","ServicoPctFilmes"] = "No"
        df2.loc[df2["ServicoSuporteOnline"] != "Yes","ServicoSuporteOnline"] = "No"
        df2["QtdeServicos"] = df2["QtdeServicos"].astype('float')
        df2.drop(columns=["TemParceiro"],inplace=True)
        #df2["vlrMedioPorServico"] = df2["CobrancaMensal"] / df2["QtdeServicos"]

        return df2  

    def getDataFrameServAdicionais(self):
        ''' gera um dataframe com as transformações "padrão"...e cria uma nova coluna apenas com o indicativo pra saber se há serviços adicionais de internet '''
        df2 = self.df.copy()
        df2["ServicosAdicionaisInternet"] = "No"
        df2.loc[df2["ServicoSegurancaOnline"] == "Yes","ServicosAdicionaisInternet"] = "Yes"
        df2.loc[df2["ServicoBackupOnline"] == "Yes","ServicosAdicionaisInternet"] =  "Yes"
        df2.loc[df2["ServicoProtecaoOnline"] == "Yes","ServicosAdicionaisInternet"] =  "Yes"
        df2.loc[df2["ServicoPctFilmes"] == "Yes","ServicosAdicionaisInternet"] = "Yes"
        df2.loc[df2["ServicoSuporteOnline"] == "Yes","ServicosAdicionaisInternet"] = "Yes"
        df2.loc[df2["ServicoTV"] == "Yes","ServicosAdicionaisInternet"] = "Yes"
        #df2.drop(columns = [ "ServicoSegurancaOnline","ServicoBackupOnline","ServicoSuporteOnline","ServicoProtecaoOnline","ServicoPctFilmes","ServicoTV"],inplace=True)
        df2.drop(columns = ["TemParceiro", "TemDependentes", "ServicoSegurancaOnline","ServicoBackupOnline","ServicoSuporteOnline","ServicoProtecaoOnline","ServicoPctFilmes","ServicoTV"],inplace=True)
        return df2

    def getDataFrameQtdeServicos(self): #80.37
        df2 = self.df.copy()
        df2["QtdeServicos"] = 0
        df2.loc[df2["TemTelefone"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["TemInternet"] != "No","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoTV"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2["QtdeServicos"] = df2["QtdeServicos"].astype('float')
        df2.drop(columns = ["TemTelefone","TemInternet","ServicoTV"],inplace=True)
        return df2          

    def getDataFrameQtdeServicos2(self): 
        df2 = self.df.copy()
        df2["QtdeServicos"] = 0
        df2["QtdeServicosAdicionais"] = 0
        df2.loc[df2["TemTelefone"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["TemInternet"] != "No","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoTV"] == "Yes","QtdeServicos"] =  df2["QtdeServicos"] + 1
        df2.loc[df2["ServicoSegurancaOnline"] == "Yes","QtdeServicosAdicionais"] =  df2["QtdeServicosAdicionais"] + 1
        df2.loc[df2["ServicoBackupOnline"] == "Yes","QtdeServicosAdicionais"] =  df2["QtdeServicosAdicionais"] + 1
        df2.loc[df2["ServicoProtecaoOnline"] == "Yes","QtdeServicosAdicionais"] =  df2["QtdeServicosAdicionais"] + 1
        df2.loc[df2["ServicoPctFilmes"] == "Yes","QtdeServicosAdicionais"] =  df2["QtdeServicosAdicionais"] + 1
        df2.loc[df2["ServicoSuporteOnline"] == "Yes","QtdeServicosAdicionais"] =  df2["QtdeServicosAdicionais"] + 1
        df2["QtdeServicos"] = df2["QtdeServicos"].astype('float')
        df2["QtdeServicosAdicionais"] = df2["QtdeServicosAdicionais"].astype('float')
        df2.drop(columns = ["TemTelefone","TemInternet","ServicoTV","ServicoSegurancaOnline","ServicoBackupOnline","ServicoSuporteOnline","ServicoProtecaoOnline","ServicoPctFilmes"],inplace=True)
        return df2          

    def trata_gender(self):
        return None

## 5. Criação dos objetos "Trata_DataFrame" para os dataframes de Teste e de Treino

In [ ]:
otdf_Treino = Trata_DataFrame(df_treino)
otdf_Teste = Trata_DataFrame(df_teste)

## 6. Recuperação dos dataframes com os tratamentos das variáveis

In [ ]:
df_treinoTratado = otdf_Treino.getDataFrameMelhoresFeatures()
df_testeTratado = otdf_Teste.getDataFrameMelhoresFeatures()
#df_testeS = objTDFTeste.getDataFrameQtdeServicos()
#df_testeA = objTDFTeste.getDataFrameServAdicionais()

In [ ]:
## Aqui, como passo intermediário, executei o modelo usando o dataframe default (getDataFrameDefault) e analisei a importância das features:
## então fui modificando o dataframe excluindo as features menos importantes...
## OBS: para utilizar, carregar antes a função treina_testa
'''
df_default = otdf_Treino.getDataFrameDefault()
preditoras = df_default.copy()
preditoras.drop(columns=["Churn","id"],inplace=True)
target = df_treinoTratado["Churn"]
categorical_features_indices = np.where(preditoras.dtypes != np.float)[0] #considerei todas as features que não são do tipo "flutuante" como categóricas
print(f"Qtde de features categóricas: {len(categorical_features_indices)}")
print(f"Colunas preditoras: {preditoras.columns}")

acc = treina_testa(mostrarFI=True)
print(f"acc: {acc}")
'''

Qtde de features categóricas: 16
Colunas preditoras: Index(['genero', 'Idoso', 'TemParceiro', 'TemDependentes', 'MesesNaCompanhia',
       'TemTelefone', 'MaisLinhas', 'TemInternet', 'ServicoSegurancaOnline',
       'ServicoBackupOnline', 'ServicoProtecaoOnline', 'ServicoSuporteOnline',
       'ServicoTV', 'ServicoPctFilmes', 'Contrato', 'ContaOnline',
       'MetodoPagamento', 'CobrancaMensal', 'CobrancaTotal'],
      dtype='object')
[(0.1219979414176875, 'genero'), (0.9463356717095784, 'Idoso'), (0.358380840616242, 'TemParceiro'), (0.7545895183322419, 'TemDependentes'), (14.482974986331126, 'MesesNaCompanhia'), (0.7144194957756642, 'TemTelefone'), (2.3906642551319375, 'MaisLinhas'), (13.01277227011774, 'TemInternet'), (4.483577047088782, 'ServicoSegurancaOnline'), (3.81270661909675, 'ServicoBackupOnline'), (1.68395081663, 'ServicoProtecaoOnline'), (2.76317943544468, 'ServicoSuporteOnline'), (1.4401108623579535, 'ServicoTV'), (2.547998080226219, 'ServicoPctFilmes'), (27.76831483017892

## 7. Separação do dataframe de treino (train.csv) entre preditoras/target

In [ ]:
preditoras = df_treinoTratado.copy()
preditoras.drop(columns=["Churn","id"],inplace=True)
target = df_treinoTratado["Churn"]
categorical_features_indices = np.where(preditoras.dtypes != np.float)[0] #considerei todas as features que não são do tipo "flutuante" como categóricas
print(f"Qtde de features categóricas: {len(categorical_features_indices)}")
print(f"Colunas preditoras: {preditoras.columns}")

Qtde de features categóricas: 9
Colunas preditoras: Index(['Idoso', 'TemDependentes', 'MesesNaCompanhia', 'MaisLinhas',
       'TemInternet', 'ServicoTV', 'ServicoPctFilmes', 'Contrato',
       'ContaOnline', 'MetodoPagamento', 'CobrancaMensal', 'CobrancaTotal',
       'QtdeServicosAdicionaisInternet'],
      dtype='object')


In [ ]:
#i_CV = 10 # Número de Cross-Validations
#i_Seed = 20111974 # semente por questões de reproducibilidade
#f_Test_Size = 0.30 # Proporção do dataframe de validação (outros valores poderiam ser 0.15, 0.20 ou 0.25)


## 8. Função para treino/teste utilizando o conjunto de treinamento (train.csv).  
### Parametros da função:
  * ts -> tamanho percentual do conjunto de teste
  * it -> parâmetro 'iterations' do CatBoostClassifier
  * lr -> parâmetro 'learning_rate' do CBC
  * depth -> parâmetro 'depth' do CBC
  * gerarArquivo -> Caso seja "True", vai aplicar o modelo no dataframe de teste (test.csv) e gerar o arquivo de resultado para submeter no kaggle
  * mostrarFI -> mostra a importância de cada feature
### Retorna: o percetual de acerto no subconjunto (teste) do dataframe de treinamento

In [ ]:
def treina_testa(ts=0.30,it=300, lr=0.03, depth=5, gerarArquivo=False, mostrarFI=False):
   X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(preditoras, target, test_size = ts)#, random_state = i_Seed)
   catb = CatBoostClassifier(iterations = it, learning_rate = lr, depth=depth, silent=True)
   catb_tuned = catb.fit(  X_treinamento, y_treinamento, cat_features=categorical_features_indices)
   y_pred = catb_tuned.predict(X_teste)
   acc_catb = round(accuracy_score(y_pred, y_teste) * 100, 2)
   #print('ts: {} it: {} lr: {} depth: {} -> '.format(ts,it,lr,depth)+'Train Data Success Score: %' + str(acc_catb))
   if (mostrarFI == True):
      l_fi = list(zip(catb_tuned.feature_importances_,X_treinamento.columns))
      print(l_fi)

   if gerarArquivo == True: 
      df_id = df_testeTratado[["id"]]
      df_teste3 = df_testeTratado.drop(columns=["id"])
      resposta = catb_tuned.predict(df_teste3)
      resposta_df = pd.DataFrame(resposta, columns=['Churn'])
      resultado_submissao = pd.concat([df_id, resposta_df],axis=1)
      resultado_submissao.head().T
      filename = 'submissao_kaggle_catb_fs_ts0{}_it{}_lr{}_depth{}_sc{}.csv'.format(round(ts*100,0),it, lr, depth,str(int(acc_catb*100)))
      resultado_submissao.to_csv(filename, index=False)   
      print(filename)
   return acc_catb   
   #result


##9. Executa a função treina_testa diversas vezes (10x), para as diferentes combinações de test_size, it, lr e depth,  armazenando o resultado de cada execução em um dicionário para análise posterior.

In [ ]:
#l_ts = [0.22, 0.24, 0.25, 0.26, 0.28, 0.30, 0.32]
#l_it =  [300, 400, 500], 
#l_lr = [0.02, 0.03, 0.04]
#l_depth = [2, 3, 4, 5]
l_ts, l_it, l_lr, l_depth = [0.26], [400], [0.02], [3]
resultado = {}

for vez in range(1,11):
   resultado[vez] = {"ts":[], "it":[], "lr":[],"depth":[],"score":[]}
   for ts in l_ts:
       print(f'execução {vez}/ts {ts}...')
       for it in l_it:
           for lr in l_lr:
               for depth in l_depth:
                  res = treina_testa(ts=ts, it=it, lr=lr, depth=depth,gerarArquivo=False, mostrarFI=False) #não vai salvar o arquivo e nem mostrar as melhores features
                  resultado[vez]["ts"].append(ts)
                  resultado[vez]["it"].append(it)
                  resultado[vez]["lr"].append(lr)
                  resultado[vez]["depth"].append(depth)
                  resultado[vez]["score"].append(res)
 

execução 1/ts 0.26...
execução 2/ts 0.26...
execução 3/ts 0.26...
execução 4/ts 0.26...
execução 5/ts 0.26...
execução 6/ts 0.26...
execução 7/ts 0.26...
execução 8/ts 0.26...
execução 9/ts 0.26...
execução 10/ts 0.26...


## 10. Transforma o dicionário gerado no passo anterior em um Dataframe e apresenta seus resultados organizados, permitindo verificar qual conjunto (ts,it, lr e depth) gerou boa média de nota e pequeno std.


In [ ]:
arrays = []
for vez, v in resultado.items():
    #print(v)
    d = pd.DataFrame(v)
    d["vez"] = vez
    arrays.append(d.copy())
dfResultado = pd.concat(arrays,axis=0,sort=False)

dfTotais = dfResultado.groupby(by=["ts","it","lr","depth"]).agg({"score":["min","max","mean","std"], "vez": ["count"]}).sort_values([("score","mean")],ascending=False)
dfTotais.head(10)

score                           vez
                       min    max    mean      std count
ts   it  lr   depth                                     
0.26 400 0.02 3      79.18  82.66  80.649  0.90407    10

## 11. Executa a função testa_treina 10 vezes, com os melhores parâmetros, gerando os arquivos.

In [ ]:
#melhor resultado:
l_it, l_lr, l_depth = [400], [0.03], [2]
l_ts = [0.26] 

resultado = {}

for vez in range(1,11):
   resultado[vez] = {"ts":[], "it":[], "lr":[],"depth":[],"score":[]}
   for ts in l_ts:
       print(f'execução {vez}/ts {ts}...')
       for it in l_it:
           for lr in l_lr:
               for depth in l_depth:
                  res = treina_testa(ts=ts, it=it, lr=lr, depth=depth,gerarArquivo=True, mostrarFI=False)
                  resultado[vez]["ts"].append(ts)
                  resultado[vez]["it"].append(it)
                  resultado[vez]["lr"].append(lr)
                  resultado[vez]["depth"].append(depth)
                  resultado[vez]["score"].append(res)
   #print(resultado[vez])   
resultado  

execução 1/ts 0.26...
submissao_kaggle_catb_fs_ts026.0_it400_lr0.03_depth2_sc8164.csv
execução 2/ts 0.26...
submissao_kaggle_catb_fs_ts026.0_it400_lr0.03_depth2_sc8000.csv
execução 3/ts 0.26...
submissao_kaggle_catb_fs_ts026.0_it400_lr0.03_depth2_sc8130.csv
execução 4/ts 0.26...
submissao_kaggle_catb_fs_ts026.0_it400_lr0.03_depth2_sc8055.csv
execução 5/ts 0.26...
submissao_kaggle_catb_fs_ts026.0_it400_lr0.03_depth2_sc8027.csv
execução 6/ts 0.26...
submissao_kaggle_catb_fs_ts026.0_it400_lr0.03_depth2_sc8143.csv
execução 7/ts 0.26...
submissao_kaggle_catb_fs_ts026.0_it400_lr0.03_depth2_sc8246.csv
execução 8/ts 0.26...
submissao_kaggle_catb_fs_ts026.0_it400_lr0.03_depth2_sc8137.csv
execução 9/ts 0.26...
submissao_kaggle_catb_fs_ts026.0_it400_lr0.03_depth2_sc8156.csv
execução 10/ts 0.26...
submissao_kaggle_catb_fs_ts026.0_it400_lr0.03_depth2_sc7993.csv


{1: {'depth': [2], 'it': [400], 'lr': [0.03], 'score': [81.64], 'ts': [0.26]},
 2: {'depth': [2], 'it': [400], 'lr': [0.03], 'score': [80.0], 'ts': [0.26]},
 3: {'depth': [2], 'it': [400], 'lr': [0.03], 'score': [81.3], 'ts': [0.26]},
 4: {'depth': [2], 'it': [400], 'lr': [0.03], 'score': [80.55], 'ts': [0.26]},
 5: {'depth': [2], 'it': [400], 'lr': [0.03], 'score': [80.27], 'ts': [0.26]},
 6: {'depth': [2], 'it': [400], 'lr': [0.03], 'score': [81.43], 'ts': [0.26]},
 7: {'depth': [2], 'it': [400], 'lr': [0.03], 'score': [82.46], 'ts': [0.26]},
 8: {'depth': [2], 'it': [400], 'lr': [0.03], 'score': [81.37], 'ts': [0.26]},
 9: {'depth': [2], 'it': [400], 'lr': [0.03], 'score': [81.57], 'ts': [0.26]},
 10: {'depth': [2], 'it': [400], 'lr': [0.03], 'score': [79.93], 'ts': [0.26]}}

In [ ]:
#a = list(zip(catb_tuned.feature_importances_,X_treinamento.columns))
#print(catb_tuned.feature_importances_)
#print(X_treinamento.columns)

